In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from Backtestingtools.electron_pnl_analyzer import PnLAnalyzer
from Backtestingtools import trade_loader
from Backtestingtools.ticks_and_books_helper import load_ticks_and_books , MarketDataAnalyzer
import datetime as dt
import pandas as pd


In [2]:
raw_data = load_ticks_and_books(["TMF", "TXF"], dt.date(2025, 3, 7), source="SRC_TAIFEX_RT_RD.ORIG_CPBQ.DEST_CPBQ")


In [3]:
market = MarketDataAnalyzer(raw_data)
events = market.detect_trade_through("TXF")

for e in events:
    print(f"{e['start_time']} ~ {e['end_time']} | {e['direction']} | {e['count']} 筆 | 平均穿價 {e['avg_diff']:.2f}")

2025-03-07 00:00:00.477841 ~ 2025-03-07 00:00:00.477841 | sell_through | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:00.477989 ~ 2025-03-07 00:00:00.477989 | buy_through | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:01.602952 ~ 2025-03-07 00:00:01.602952 | sell_through | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:01.727967 ~ 2025-03-07 00:00:01.727967 | buy_through | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:06.602949 ~ 2025-03-07 00:00:06.602949 | buy_through | 2 筆 | 平均穿價 1.50
2025-03-07 00:00:11.352779 ~ 2025-03-07 00:00:11.352779 | buy_through | 2 筆 | 平均穿價 1.50
2025-03-07 00:00:22.102832 ~ 2025-03-07 00:00:22.102832 | buy_through | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:29.602804 ~ 2025-03-07 00:00:29.602804 | sell_through | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:39.727846 ~ 2025-03-07 00:00:39.727846 | buy_through | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:43.228136 ~ 2025-03-07 00:00:43.228159 | sell_through | 2 筆 | 平均穿價 1.50
2025-03-07 00:00:57.852944 ~ 2025-03-07 00:00:57.852944 | sell_through | 1 筆 | 平均穿價 1.00
2025-03-07 00:01:41.477957 

In [6]:
df_tick=raw_data['TXF']['tick']

In [5]:
def detect_trade_through_from_tick(df_tick, merge_window_ms=1000):
    events = []
    for t, row in df_tick.iterrows():
        px = row.get("px")
        bid = row.get("bid_px")
        ask = row.get("ask_px")

        if pd.isna(px) or bid == 0 or ask == 0:
            continue

        if px > ask:
            events.append((t, "buy", px - ask))
        elif px < bid:
            events.append((t, "sell", bid - px))

    if not events:
        return []

    merged = []
    group = [events[0]]

    for curr in events[1:]:
        prev = group[-1]
        time_gap = (curr[0] - prev[0]).total_seconds() * 1000
        if curr[1] == prev[1] and time_gap <= merge_window_ms:
            group.append(curr)
        else:
            merged.append(group)
            group = [curr]

    merged.append(group)

    results = []
    for g in merged:
        start = g[0][0]
        end = g[-1][0]
        direction = g[0][1]
        count = len(g)
        avg_diff = sum(x[2] for x in g) / count
        results.append({
            "start_time": start,
            "end_time": end,
            "direction": direction,
            "count": count,
            "avg_diff": avg_diff
        })

    return results


In [8]:
df_tick

,ex_time,instr,sz,px,bid_px,ask_px,bid_sz,ask_sz
time,,,,,,,,
2025-03-07 00:00:00.227848,2025-03-07 00:00:00.136,FUT_TAIFEX_TXF:202503,2,22655.0,22653.0,22655.0,5,2
2025-03-07 00:00:00.477841,2025-03-07 00:00:00.389,FUT_TAIFEX_TXF:202503,1,22655.0,22655.0,22656.0,1,2
2025-03-07 00:00:00.477841,2025-03-07 00:00:00.389,FUT_TAIFEX_TXF:202503,1,22654.0,22655.0,22656.0,1,2
2025-03-07 00:00:00.477961,2025-03-07 00:00:00.405,FUT_TAIFEX_TXF:202503,1,22656.0,22655.0,22656.0,1,2
2025-03-07 00:00:00.477975,2025-03-07 00:00:00.412,FUT_TAIFEX_TXF:202503,1,22656.0,22655.0,22656.0,1,2
...,...,...,...,...,...,...,...,...
2025-03-07 23:59:54.326716,2025-03-07 23:59:54.245,FUT_TAIFEX_TXF:202503,1,22436.0,22435.0,22436.0,3,2
2025-03-07 23:59:54.326740,2025-03-07 23:59:54.245,FUT_TAIFEX_TXF:202503,1,22436.0,22435.0,22436.0,3,2
2025-03-07 23:59:55.076760,2025-03-07 23:59:55.050,FUT_TAIFEX_TXF:202503,1,22437.0,22436.0,22437.0,2,1


In [ ]:
events = detect_trade_through_-03-07 22:26:37.451830 from_tick(df_tick)

for e in events:
    print(f"{e['start_time']} ~ {e['end_time']} | {e['direction']} | {e['count']} 筆 | 平均穿價 {e['avg_diff']:.2f}")


2025-03-07 00:00:00.477841 ~ 2025-03-07 00:00:00.477841 | sell | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:00.477989 ~ 2025-03-07 00:00:00.477989 | buy | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:01.602952 ~ 2025-03-07 00:00:01.602952 | sell | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:01.727967 ~ 2025-03-07 00:00:01.727967 | buy | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:06.602949 ~ 2025-03-07 00:00:06.602949 | buy | 2 筆 | 平均穿價 1.50
2025-03-07 00:00:11.352779 ~ 2025-03-07 00:00:11.352779 | buy | 2 筆 | 平均穿價 1.50
2025-03-07 00:00:22.102832 ~ 2025-03-07 00:00:22.102832 | buy | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:29.602804 ~ 2025-03-07 00:00:29.602804 | sell | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:39.727846 ~ 2025-03-07 00:00:39.727846 | buy | 1 筆 | 平均穿價 1.00
2025-03-07 00:00:43.228136 ~ 2025-03-07 00:00:43.228159 | sell | 2 筆 | 平均穿價 1.50
2025-03-07 00:00:57.852944 ~ 2025-03-07 00:00:57.852944 | sell | 1 筆 | 平均穿價 1.00
2025-03-07 00:01:41.477957 ~ 2025-03-07 00:01:41.477957 | buy | 1 筆 | 平均穿價 1.00
2025-03-07 00:01:44.352828 ~ 2025-0

In [ ]:
market = MarketDataAnalyzer(raw_data)

In [ ]:
def query_book_between(df_book, timestamp, window_ms=1000):
    """
    查詢 timestamp 前後 window_ms 毫秒內的五檔資料 snapshot。
    """
    if df_book.empty:
        print("⚠ df_book 是空的")
        return df_book

    if not isinstance(df_book.index, pd.DatetimeIndex):
        df_book.index = pd.to_datetime(df_book.index)

    start = timestamp - pd.Timedelta(milliseconds=window_ms)
    end = timestamp + pd.Timedelta(milliseconds=window_ms)

    df_window = df_book.loc[(df_book.index >= start) & (df_book.index <= end)]
    print(f"查詢範圍：{start} ~ {end}")
    print(f"找到 {len(df_window)} 筆資料")

    return df_window


In [ ]:
# 設定某個成交時間（timestamp）
some_time = pd.Timestamp("2025-03-04 21:30:02.123")

# 直接測試
df_window = query_book_between(raw_data['TMF']['book'], some_time, window_ms=1000)

# 顯示結果
print(df_window.head())
